Cargar modulos

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

Crear sesión de spark

In [18]:
spark= SparkSession.builder.appName("Mi primera app").getOrCreate()

In [19]:
spark

Cargar datos

In [1]:
import os
import zipfile
import pathlib

#with zipfile.ZipFile(".\\.\\Datos\\Combined_Flights_2018.zip", mode="r") as archivo:
#    archivo.printdir()

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.zip')

path_datos_csv = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.csv')

        
with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
    archivo_zip.extract('Combined_Flights_2018.csv', path=pathlib.Path.joinpath(direccion.parent.parent,'Datos'))
